In [ ]:
import requests
import json
import quandl #https://docs.quandl.com/docs/python-time-series give a how to guide
from config import q_api_key
from config import y_api_key
from config import c_api_key

import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
c = Census(c_api_key, year=2017)

import gmaps
from config import gkey

gmaps.configure(api_key=gkey)

from uszipcode import SearchEngine, SimpleZipcode, Zipcode #https://uszipcode.readthedocs.io/index.html
search = SearchEngine()

In [ ]:
quandl.ApiConfig.api_key = q_api_key

In [ ]:
zipcode_df=pd.read_csv("../Resources/zipcode.csv")
zipcode_df['Zipcode'] = pd.to_numeric(zipcode_df['Zipcode'])
zipcode_df.head()

In [ ]:
zipcodes = zipcode_df['Zipcode'].tolist()
medianrent=[]
medianhomevalues=[]
for x in range(0,len(zipcodes)):
    try:
        mr_raw=quandl.get(f"ZILLOW/Z{zipcodes[x]}_ZRIAH", collapse="annual", start_date="2018-12-31", end_date="2018-12-31")
        mr_list=mr_raw["Value"].tolist()
        medianrent.append(mr_list[0])
        print(f'Pulled rent data for {zipcodes[x]}')
    except:
        mr_missing='NaN'
        medianrent.append(mr_missing)
        print(f'Rent data for {zipcodes[x]} not found')
    try:
        mhv_raw=quandl.get(f"ZILLOW/Z{zipcodes[x]}_ZHVIAH", collapse="annual", start_date="2018-12-31", end_date="2018-12-31")
        mhv_list=mhv_raw["Value"].tolist()
        medianhomevalues.append(mhv_list[0])
        print(f'Pulled home value data for {zipcodes[x]}')
    except:
        mhv_missing='NaN'
        medianhomevalues.append(mhv_missing)
        print(f'Home value data for {zipcodes[x]} not found')
print(medianrent)
print(medianhomevalues)

In [ ]:
housing_df = pd.DataFrame({"Zipcode":zipcodes, "Median Rent":medianrent, "Median Home Value":medianhomevalues})
housing_df.head()

In [ ]:
rent_df = housing_df[["Zipcode", "Median Rent"]]
rent_df.head()

In [ ]:
headers = {'Authorization': 'Bearer %s' % y_api_key}

categories = 'nightlife' 
url='https://api.yelp.com/v3/businesses/search'
category_l = []
for zipcode in zipcodes:
    print(f'starting datapull for {zipcode}')    
    yelp_data = []    
    for offset in range(0, 1000, 50):
        params = {
            'limit': 50, 
            'location': zipcode,
            'categories': categories,
            'offset': offset
        }

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:            
            yelp_data += response.json()['businesses']
        elif response.status_code == 400:
            print('400 Bad Request')
            break
    print(f'finished datapull for {zipcode}')    
    zipcode_b =[]    
    for business in range(0,len(yelp_data)):
        currentzip =str(yelp_data[business]["location"]["zip_code"])
        if currentzip == str(zipcode):
            zipcode_b.append(currentzip)            
    numberinzip=len(zipcode_b)
    category_l.append(numberinzip)
    print(f'new appended category total: {numberinzip}  for {zipcode}')

In [ ]:
categories = 'restaurants' 
url='https://api.yelp.com/v3/businesses/search'
category_r = []
for zipcode in zipcodes:
    print(f'starting datapull for {zipcode}')   
    yelp_data = []    
    for offset in range(0, 1000, 50):
        params = {
            'limit': 50, 
            'location': zipcode,
            'categories': categories,
            'offset': offset
        }

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:            
            yelp_data += response.json()['businesses']
        elif response.status_code == 400:
            print('400 Bad Request')
            break
    print(f'finished datapull for {zipcode}')    
    zipcode_b =[]   
    for business in range(0,len(yelp_data)):
        currentzip =str(yelp_data[business]["location"]["zip_code"])
        if currentzip == str(zipcode):
            zipcode_b.append(currentzip)           
    numberinzip=len(zipcode_b)
    category_r.append(numberinzip)
    print(f'new appended category total: {numberinzip}  for {zipcode}')

In [ ]:
categories = 'grocery' 
url='https://api.yelp.com/v3/businesses/search'
category_g = []
for zipcode in zipcodes:
    print(f'starting datapull for {zipcode}')   
    yelp_data = []    
    for offset in range(0, 1000, 50):
        params = {
            'limit': 50, 
            'location': zipcode,
            'categories': categories,
            'offset': offset
        }

        response = requests.get(url, headers=headers, params=params)
        if response.status_code == 200:           
            yelp_data += response.json()['businesses']
        elif response.status_code == 400:
            print('400 Bad Request')
            break
    print(f'finished datapull for {zipcode}')   
    zipcode_b =[]   
    for business in range(0,len(yelp_data)):
        currentzip =str(yelp_data[business]["location"]["zip_code"])
        if currentzip == str(zipcode):
            zipcode_b.append(currentzip)            
    numberinzip=len(zipcode_b)
    category_g.append(numberinzip)
    print(f'new appended category total: {numberinzip}  for {zipcode}')

In [ ]:
yelp_df=pd.DataFrame({"Zipcode":zipcodes, "Total Nightlife": category_l, "Total Restaurant": category_r, 
                      "Total Grocery": category_g})

In [ ]:
yelp_df.to_csv("../Output/Yelp_Data.csv", encoding="utf-8", index=False)

In [ ]:
df_crime = pd.read_csv('../Resources/Crimes_2018.csv')
df_crime_latlon=df_crime.loc[:,['Latitude', 'Longitude']]
df_crime_latlon.head()

In [ ]:
len(df_crime_latlon)

In [ ]:
from uszipcode import SearchEngine, SimpleZipcode, Zipcode
search = SearchEngine()

In [ ]:
res = []
for index, row in df_crime_latlon.iterrows():    
    result = search.by_coordinates(row["Latitude"], row["Longitude"], radius=3, returns=1)    
    res.append(result)

In [ ]:
zipcodes_crime = []
for i in range(len(res)):    
    for x in res[i]:       
        zip = x.zipcode     
        zipcodes_crime.append(zip)
len(zipcodes_crime)

In [ ]:
crime_zip_df = pd.DataFrame({"Zipcode": zipcodes_crime})

In [ ]:
zipcode_count = crime_zip_df["Zipcode"].value_counts().reset_index()
crime_count = zipcode_count.rename(columns={"index": "Zipcode","Zipcode":"Crime Count"})

crime_count['Zipcode'] = pd.to_numeric(crime_count['Zipcode'])
crime_count_df = crime_count.sort_values("Zipcode")
crime_count_df

In [ ]:
crime_count_df.to_csv("../Output/Crime_Count_by_Zip.csv", encoding="utf-8", index=False)

In [ ]:
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E"), {'for': 'zip code tabulation area:*'})



census_pd = pd.DataFrame(census_data)

   

census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "NAME": "Name", "zip code tabulation area": "Zipcode"})
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)
census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate"]]
census_pd['Zipcode'] = pd.to_numeric(census_pd['Zipcode'])
census_pd.head()

In [ ]:
chicago_census_df = pd.merge(zipcode_df, census_pd, on="Zipcode")
chicago_census_df.head()

In [ ]:
chicago_census_df["Lat"] = ""
chicago_census_df["Lng"] = ""
chicago_census_df.head()

In [ ]:
params = {"key": gkey}

for index, row in chicago_census_df.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    address = row['Zipcode']
    
    params['address'] = f"{address}"
    
    response = requests.get(base_url, params=params).json()  
  
    

    chicago_census_df.loc[index, "Lat"] = response["results"][0]["geometry"]["location"]["lat"]
    chicago_census_df.loc[index, "Lng"] = response["results"][0]["geometry"]["location"]["lng"]

chicago_census_df.head()

In [ ]:
chicago_housing_census_df =  pd.merge(housing_df, chicago_census_df, on="Zipcode")
chicago_housing_census_df.head()

In [ ]:
chicago_housing_census_df.to_csv("../Output/Chicago_Housing_and_Census_Data.csv", encoding="utf-8", index=False)

In [ ]:
airbnb_df = pd.read_csv('../Resources/AisBnB_listings.csv')
airbnb_df.head()

In [ ]:
airbnb_cleaned = airbnb_df[["name", "neighbourhood","latitude", "longitude", "number_of_reviews"]]
airbnb_cleaned.head()

In [ ]:
results = []
for index, row in airbnb_cleaned.iterrows():   
    result = search.by_coordinates(row["latitude"], row["longitude"], radius=3, returns=1)    
    results.append(result)

In [ ]:
len(results)

In [ ]:
zipcodes_airbnb= []
for i in range(len(results)):    
    for x in results[i]:        
        zip = x.zipcode     
        zipcodes_airbnb.append(zip)
len(zipcodes_airbnb)
zipcodes_airbnb

In [ ]:
airbnb_zip_df = pd.DataFrame({"Zipcode": zipcodes_airbnb})

In [ ]:
airbnb_data_by_zipcode = airbnb_cleaned.join(airbnb_zip_df)
airbnb_data_by_zipcode.head()

In [ ]:
airbnb_zipcode = airbnb_data_by_zipcode[["Zipcode", "number_of_reviews"]]
airbnb_zipcode.head()

In [ ]:
airbnb_zipcode_sort = airbnb_zipcode.sort_values("Zipcode")
airbnb_zipcode_sort.head()

In [ ]:
airbnb_zipcode_sort["Zipcode"].unique()

In [ ]:
grouped_zipcode = airbnb_zipcode_sort.groupby(['Zipcode'])

In [ ]:
avg_reviews = round(grouped_zipcode.mean()["number_of_reviews"])

airbnb_reviews_zipcode = pd.DataFrame({"AirBnB Review Count": avg_reviews})
airbnb_reviews_zipcode.head()

In [ ]:
airbnb_reviews_zipcode.to_csv("../Output/AirBnB_Reviews_Zipcode.csv", encoding="utf-8")

In [ ]:
airbnb_zipcode_df = pd.read_csv("../Output/AirBnB_Reviews_Zipcode.csv")

In [ ]:
airbnb_rent_df = pd.merge(airbnb_zipcode_df, rent_df, on="Zipcode")
airbnb_rent_df

In [ ]:
airbnb_rent_df.to_csv("../Output/AirBnB_Reviews_Rent_Data.csv", encoding="utf-8", index=False)